In [2]:
# 制造最初表格
import numpy as np
import pandas as pd

import os
os.chdir(r'D:\paper\rowdata')

code_ls = []
with open("Code.txt", 'r') as code:
    for line in code:
        code_ls.append(line[:-1])

date_ls = ['2009-12-31', '2010-12-31', '2011-12-31',
          '2012-12-31', '2013-12-31', '2014-12-31',
          '2015-12-31', '2016-12-31', '2017-12-31',
          '2018-12-31', '2019-12-31', '2020-12-31',
          '2021-12-31',]

n_1 = len(code_ls)
n_2 = len(date_ls)

Code_ls = []
for code in code_ls:
    for i in range(n_2):
        Code_ls.append(code)

Time_ls = []
for i in range(n_1):
    for date in date_ls:
        Time_ls.append(date)

df = pd.DataFrame({"Code": Code_ls, "Time": Time_ls})
df["Time"] = df["Time"].astype('datetime64')

In [3]:
# 被解释变量 -- 违规与否
os.chdir(r'D:\paper\rowdata\violate')

violate = pd.read_csv('violate_clean.csv', parse_dates=[1],
                      infer_datetime_format=True, dtype={"Code": str})
violate = violate.rename(columns={"Time_a": "Time"})

violate["Vio"] = [1 if i == 'Y' else 0 for i in violate["Vio"]]
violate["Vio_wide"] = [1 if i == 'Y' else 0 for i in violate["Vio_wide"]]
df = pd.merge(df, violate, on=['Code', 'Time'], how='outer')

In [8]:
# 资产负债表
os.chdir(r'D:\paper\rowdata\balance')
balance = pd.read_csv("balance_clean.csv", dtype={"Code": str, "Time":str})

qtysk = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][2]
            qtysk.append(q)
        except:
            qtysk.append(np.nan)
df["Qtysk"] = qtysk

qtyfk = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][5]
            qtyfk.append(q)
        except:
            qtyfk.append(np.nan)
df["Qtyfk"] = qtyfk

zcze = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][4]
            zcze.append(q)
        except:
            zcze.append(np.nan)
df["Zcze"] = zcze
df["Zczelog"] = [np.log(i) if i != 0 else np.nan for i in df["Zcze"]]

In [31]:
qtyskzj = []
for code in code_ls:
    q = list(df[df["Code"]==code]["Qtysk"].shift(-1)
             - df[df["Code"]==code]["Qtysk"])
    qtyskzj = qtyskzj + q
df["Qtyskzj"] = qtyskzj

df["Zjzy"] = df["Qtysk"] - df["Qtyfk"]

In [33]:
df["Zjzy_t"] = df["Zjzy"] / df["Zcze"]
df["Qtyskzj_t"] = df["Qtyskzj"] / df["Zcze"]

In [34]:
# 解释变量
os.chdir(r'D:\paper\rowdata\rzrq')

rzrq = pd.read_csv('rzrq_clean1.csv', dtype={"Code": str})

def treated_detect(code, df_t):
    try:
        if df_t[df_t["Code"]==code]["Treated"].iloc[0] == 1:
            if df_t[df_t["Code"]==code]["Tc_t"].iloc[0] == 0:
                return 1
            else:
                return 0
        else:
            return 0
    except:
        return 0

df["Treated"] = [treated_detect(i, rzrq) for i in df["Code"]]

def treated_detect_wide(code, df_t):
    try:
        if df_t[df_t["Code"]==code]["Treated"].iloc[0] == 1:
            return 1
        else:
            return 0
    except:
        return 0
    
df["Treated_wide"] = [treated_detect_wide(i, rzrq) for i in df["Code"]]

def post_detect(code, df):
    if df[df["Code"]==code]["Treated"].iloc[0] == 0:
        return [0 for i in range(13)]
    else:
        # 获取入选年度，若入选年度为2010，结果为10，如入选年度为2018，结果为18
        in_year = int(df[df["Code"]==code]["Time"].iloc[0][:4]) - 2000
        if df[df["Code"]==code]["Tc_t"].iloc[0] == 0:
            n_1 = in_year - 9
            n_2 = 13 - n_1
            ls = [0 for i in range(n_1)] + [1 for i in range(n_2)]
        else:
            # 获取被剔除年度
            out_year = int(df[df["Code"]==code]["Time_a"].iloc[0][:4]) - 2000
            n_1 = in_year - 9
            n_2 = out_year - in_year
            n_3 = 21 - out_year + 1
            ls = [0 for i in range(n_1)] + [1 for i in range(n_2)] + [0 for i in range(n_3)]
    return ls

post = []
for code in code_ls:
    post = post + post_detect(code, rzrq)
df["Post"] = post

df["Rzrq"] = df["Treated"] * df["Post"]

In [35]:
df["Time"] = df["Time"].astype(str)

os.chdir(r'D:\paper\rowdata\rzrq_v')

rzrq_j = pd.read_csv("rzrq_j_clean.csv", dtype={"Stockcode": str})
rzrq_j = rzrq_j.rename(columns={"Stockcode": "Code"})

rzmr = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            rzmr.append(p)
        except:
            rzmr.append(0)
df["Rzmr"] = rzmr

rzye = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            rzye.append(p)
        except:
            rzye.append(0)
df["Rzye"] = rzye

rqmc = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            rqmc.append(p)
        except:
            rqmc.append(0)
df["Rqmc"] = rqmc

rqye = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            rqye.append(p)
        except:
            rqye.append(0)
df["Rqye"] = rqye

In [41]:
os.chdir(r'D:\paper\rowdata\ltsz')
df["Time"] = df["Time"].astype('datetime64')
ltsz = pd.read_csv('ltsz_clean.csv', parse_dates=[1],
                  dtype={"Code": str})
ltsz = ltsz.rename(columns={"ltsz": "Ltsz"})
df = pd.merge(df, ltsz, on=['Code', 'Time'], how='outer')
df["Ltszlog"] = [np.log(i) if i>0 else np.nan for i in df["Ltsz"]]

In [44]:
# 财务指标
os.chdir(r'D:\paper\rowdata\cwzb')
cwzb = pd.read_csv("cwzb.csv", dtype={"Code":str})

zcfz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            zcfz.append(p)
        except:
            zcfz.append(np.nan)
df["Zcfz"] = [i if i >= 0 else np.nan for i in zcfz]

yszz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            yszz.append(p)
        except:
            yszz.append(np.nan)
df["Yszz"] = yszz

roa = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            roa.append(p)
        except:
            roa.append(np.nan)
df["Roa"] = roa

tobin = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][6]
            tobin.append(p)
        except:
            tobin.append(np.nan)
df["TobinQ"] = tobin

zmszb = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][7]
            zmszb.append(p)
        except:
            zmszb.append(np.nan)
df["Zmszb"] = zmszb

In [48]:
# 市场数据
os.chdir(r'D:\paper\rowdata\scsj')
scsj = pd.read_csv("scsj.csv", dtype={"Code": str})
scsj["Time"] = [i[:4] + "-12-31" for i in scsj["Time"]]

hbl = []
for code in code_ls:
    for date in date_ls:
        try:
            q = scsj.loc[scsj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            hbl.append(p)
        except:
            hbl.append(np.nan)
df["Hbl"] = hbl

hsl = []
for code in code_ls:
    for date in date_ls:
        try:
            q = scsj.loc[scsj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            hsl.append(p)
        except:
            hsl.append(np.nan)
df["Hsl"] = hsl

In [54]:
# 持股数据
os.chdir(r'D:\paper\rowdata\holdings')
holdings = pd.read_csv('holdings_clean.csv', dtype={"Code": str})
first = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            first.append(p)
        except:
            first.append(np.nan)
first = np.array(first)
first = first / 100
df["First"] = first

two_ten = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            two_ten.append(p)
        except:
            two_ten.append(np.nan)
df["Two_ten"] = two_ten
df["Two_ten"] = df["Two_ten"] / 100

gqzhd = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            gqzhd.append(p)
        except:
            gqzhd.append(np.nan)
df["Gqzhd"] = gqzhd

jgtzz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            jgtzz.append(p)
        except:
            jgtzz.append(np.nan)
jgtzz = np.array(jgtzz)
jgtzz = jgtzz / 100
df["Jgtzz"] = jgtzz

lzhy = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][6]
            lzhy.append(int(p))
        except:
            lzhy.append(np.nan)
df["Lzhy"] = lzhy

glccg = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][7]
            glccg.append(p)
        except:
            glccg.append(np.nan)
df["Glccg"] = glccg

dshgm = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][8]
            dshgm.append(p)
        except:
            dshgm.append(np.nan)
df["Dshgm"] = dshgm
df["Dshgmlog"] = [np.log(i) if i > 0 else np.nan for i in df["Dshgm"]]

ddzb = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][9]
            ddzb.append(p)
        except:
            ddzb.append(np.nan)
ddzb = np.array(ddzb)
ddzb = ddzb / 100
df["Ddzb"] = ddzb

In [63]:
# 外部环境
os.chdir(r'D:\paper\rowdata\wbhj')
wbhj = pd.read_csv("wbhj.csv", dtype={"Code": str})
fxszz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = wbhj.loc[wbhj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            fxszz.append(p)
        except:
            fxszz.append(np.nan)
df["Fxszz"] = fxszz
df["Fxslg"] = [np.log(1+i) if i >= 0 else np.nan for i in df["Fxszz"]]

big4 = []
for code in code_ls:
    for date in date_ls:
        try:
            q = wbhj.loc[wbhj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            big4.append(p)
        except:
            big4.append(np.nan)
big4_ls = []
for i in big4:
    if i == 1:
        big4_ls.append(int(1))
    elif i == 2:
        big4_ls.append(int(0))
    else:
        big4_ls.append(np.nan)

df["Sjjg"] = big4_ls

In [69]:
# 基本数据 bascis
os.chdir(r'D:\paper\rowdata\basics')
basics = pd.read_csv("basics_clean.csv", parse_dates = [3,4], infer_datetime_format=True,
                    dtype={"Code": str})
industry = []
industry_d = []
for code in code_ls:
    for date in date_ls:
        try:
            p = basics.loc[basics["Code"]==code].values[0][2]
            industry.append(p[0])
            industry_d.append(p)
        except:
            industry.append(np.nan)
            industry_d.append(np.nan)
df["Industry"] = industry
df["Industry_d"] = industry_d

basics["Estbdt"] = basics["Estbdt"].astype(str)
basics["Listdt"] = basics["Listdt"].astype(str)
qynl = []
for code in code_ls:
    for date in date_ls:
        try:
            p = int(basics.loc[basics["Code"]==code].values[0][3][:4])
            p = int(date[:4]) - p
            qynl.append(p)
        except:
            qynl.append(np.nan)
df["Qynl"] = qynl

ssnx = []
for code in code_ls:
    for date in date_ls:
        try:
            p = int(basics.loc[basics["Code"]==code].values[0][4][:4])
            p = int(date[:4]) - p
            ssnx.append(p)
        except:
            ssnx.append(np.nan)
df["Ssnx"] = ssnx

cqxz = []
for code in code_ls:
    for date in date_ls:
        try:
            p = basics.loc[basics["Code"]==code].values[0][7]
            cqxz.append(p)
        except:
            cqxz.append(np.nan)
df["Cqxz"] = [1 if i in ['P0301', 'P0307', 'P0308'] else 0 for i in cqxz]

In [76]:
df["Ssnxlog"] = [np.log(1+i) if i>= 0 else np.nan for i in df["Ssnx"]]

In [89]:
# 行业TobinQ
os.chdir(r'D:\paper\rowdata\Hyt')
hyt = pd.read_csv("hyt.csv", dtype={"Time":str})

hytq = []
n = len(df["Code"])
for i in range(n):
    time = str(df.iloc[i, :].values[1])[:10]
    ind = df.iloc[i, :].values[44]
    try:
        qq = hyt[hyt["Ind"]==ind]
        qq = qq[qq["Time"]==time].values[0][2]
        hytq.append(qq)
    except:
        hytq.append(np.nan)

df["HTobinQ"] = hytq

In [94]:
# HHI 赫芬达尔指数
os.chdir(r'D:\paper\rowdata')
hhi = pd.read_csv("HHI_clean.csv", dtype={"Time":str})

hhi_ls = []
n = len(df["Code"])
for i in range(n):
    time = str(df.iloc[i, :].values[1])[:10]
    ind = df.iloc[i, :].values[44]
    try:
        qq = hhi[hhi["Ind"]==ind]
        qq = qq[qq["Time"]==time].values[0][2]
        hhi_ls.append(qq)
    except:
        hhi_ls.append(np.nan)

df["HHI"] = hhi_ls

In [117]:
# 利润表
os.chdir(r'D:\paper\rowdata\income')
income = pd.read_csv("income_clean.csv", dtype={"Code": str, "Time" :str})

yylr = []
jlr = []
n = len(df["Code"])
for i in range(n):
    time = str(df.iloc[i, :].values[1])[:10]
    code = df.iloc[i, :].values[0]
    try:
        qq = income[income["Code"]==code]
        ss = qq[qq["Time"]==time].values[0][4]
        pp = qq[qq["Time"]==time].values[0][3]
        jlr.append(ss)
        yylr.append(pp)
    except:
        jlr.append(np.nan)
        yylr.append(np.nan)

df["Yylr"] = yylr
df["Jlr"] = jlr

# 储存数据

In [123]:
# 储存原始数据
os.chdir(r'D:\paper\rowdata\1')

df.to_csv("row_data.csv", index=False)

In [124]:
df.dropna()

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Industry,Industry_d,Qynl,Ssnx,Cqxz,Ssnxlog,HTobinQ,HHI,Yylr,Jlr
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,K,K70,21,18,1,2.944439,1.293807,0.075733,2.873544e+09,2.874475e+09
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,K,K70,22,19,1,2.995732,1.059200,0.054805,3.075290e+09,3.083108e+09
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,K,K70,23,20,1,3.044522,0.626153,0.063567,4.366492e+09,4.372887e+09
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,K,K70,24,21,1,3.091042,0.668239,0.071580,5.607606e+09,5.613873e+09
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,K,K70,25,22,1,3.135494,0.712191,0.076325,7.791416e+09,7.795896e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37203,603998,2019-12-31,0,1,0,1,2.808119e+08,2.524324e+08,1.793355e+09,21.307354,...,C,C27,19,5,0,1.791759,4.721329,0.034775,1.039226e+08,9.101487e+07
37204,603998,2020-12-31,0,0,0,0,3.424648e+08,2.202338e+08,1.973600e+09,21.403125,...,C,C27,20,6,0,1.945910,6.798180,0.035581,9.372988e+07,8.414300e+07
37213,603999,2016-12-31,0,0,0,0,2.510770e+08,6.391116e+06,1.751648e+09,21.283823,...,R,R85,7,1,1,0.693147,2.069841,0.093014,7.678633e+07,8.544740e+07
37214,603999,2017-12-31,0,0,0,0,2.581941e+08,7.733190e+06,1.809583e+09,21.316362,...,R,R85,8,2,1,1.098612,1.910157,0.076465,8.422745e+07,9.152340e+07
